# # Modelagem 1-dim

In [1]:
import keras as K
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from keras import layers
from sklearn.model_selection import train_test_split

%matplotlib inline

In [4]:
DIR_DATAFRAMES = '/media/hbrandao/STORAGE_HDD_1TB/PPGI/BRAVO-DATAFRAMES/DATAFRAMES'
DIR_MFCC = '/media/hbrandao/STORAGE_HDD_1TB/PPGI/BRAVO-DATAFRAMES/MFCC'

In [5]:
!ls -lah $DIR_DATAFRAMES

total 4,1G
drwxrwxrwx 1 hbrandao hbrandao 4,0K mar 21 23:15 .
drwxrwxrwx 1 hbrandao hbrandao  248 mar 21 23:49 ..
-rwxrwxrwx 1 hbrandao hbrandao 181K mar 21 23:27 Xfusao.csv
-rwxrwxrwx 1 hbrandao hbrandao 1,3G mar 21 23:28 Xsinal_0padded.npy
-rwxrwxrwx 1 hbrandao hbrandao 372M mar 21 23:28 Xsinal.npy
-rwxrwxrwx 1 hbrandao hbrandao 1,3G mar 21 23:28 Xsinal_stretch_0padded.npy
-rwxrwxrwx 1 hbrandao hbrandao 1,3G mar 21 23:28 Xsinal_tile_0padded.npy


In [6]:
!ls -lahR $DIR_MFCC

/media/hbrandao/STORAGE_HDD_1TB/PPGI/BRAVO-DATAFRAMES/MFCC:
total 0
drwxrwxrwx 1 hbrandao hbrandao 320 mar 22 01:49 .
drwxrwxrwx 1 hbrandao hbrandao 248 mar 21 23:49 ..
drwxrwxrwx 1 hbrandao hbrandao 440 mar 22 01:55 128
drwxrwxrwx 1 hbrandao hbrandao 440 mar 22 01:54 64
drwxrwxrwx 1 hbrandao hbrandao 304 mar 22 01:53 mean

/media/hbrandao/STORAGE_HDD_1TB/PPGI/BRAVO-DATAFRAMES/MFCC/128:
total 944M
drwxrwxrwx 1 hbrandao hbrandao  440 mar 22 01:55 .
drwxrwxrwx 1 hbrandao hbrandao  320 mar 22 01:49 ..
-rwxrwxrwx 1 hbrandao hbrandao 315M mar 22 02:10 Xsinal_pad_128mfcc.npy
-rwxrwxrwx 1 hbrandao hbrandao 315M mar 22 02:10 Xsinal_stretch_128mfcc.npy
-rwxrwxrwx 1 hbrandao hbrandao 315M mar 22 02:11 Xsinal_tile_128mfcc.npy

/media/hbrandao/STORAGE_HDD_1TB/PPGI/BRAVO-DATAFRAMES/MFCC/64:
total 472M
drwxrwxrwx 1 hbrandao hbrandao  440 mar 22 01:54 .
drwxrwxrwx 1 hbrandao hbrandao  320 mar 22 01:49 ..
-rwxrwxrwx 1 hbrandao hbrandao 158M mar 22 02:05 Xsinal_pad_64mfcc.npy
-rwxrwxrwx 1 hbrandao hbra

In [ ]:
PHYS_DEVICE_NAME = tf.config.list_physical_devices('GPU')[0].name[-5:]
if PHYS_DEVICE_NAME:
    print(PHYS_DEVICE_NAME)
else:
    raise Exception('GPU não encontrada')

## Dados

In [ ]:
!ls DATAFRAMES/

Iremos organizar os conjuntos a serem utilizados, onde:

- $X$ é formado pelos dados de entrada, que representam os arquivos de áudio;
- $Y$ é formado pelas emoções;
- $Z$ é formado pelas intensidades

In [ ]:
X = pd.read_csv('DATAFRAMES/Xmfcc.csv', sep=';')
X.shape

In [ ]:
X.head()

In [ ]:
Y = pd.read_csv('DATAFRAMES/Xfusao.csv', sep=';')
Y.drop(labels=['path', 'pessoa', 'genero', 'duracao'], axis=1, inplace=True)
Y.shape

In [ ]:
Y.tail()

In [ ]:
Y['emocao'].value_counts()

In [ ]:
Y['intensidade'].value_counts()

In [ ]:
DE_PARA_EMOCAO = {
    'alegria': np.array([1,0,0,0]),
    'medo': np.array([0,1,0,0]),
    'raiva': np.array([0,0,1,0]),
    'surpresa': np.array([0,0,0,1]),
}

DE_PARA_INTENSIDADE = {
    'fraca': np.array([1,0,0,0]),
    'moderada': np.array([0,1,0,0]),
    'forte': np.array([0,0,1,0]),
    'pico': np.array([0,0,0,1]),
}

def encoding_emocao(emocao: str):
    return DE_PARA_EMOCAO.get(emocao)

def encoding_intensidade(intensidade: str):
    return DE_PARA_INTENSIDADE.get(intensidade)

In [ ]:
Y['enc emocao'] = Y['emocao'].apply(encoding_emocao)

In [ ]:
Y['enc intensidade'] = Y['intensidade'].apply(encoding_intensidade)  # que é nosso Z

In [ ]:
Y.tail()

## Data Frame

In [ ]:
df = pd.concat(objs=[Y, X], axis=1)
df.drop(labels=['emocao', 'intensidade'], axis=1, inplace=True)
df.shape

In [ ]:
del X, Y

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
Xfeatures = df.values[:,4:].astype('float32')

Xfeatures.shape

In [ ]:
a = df['arquivo'].iloc[777]

plt.plot(Xfeatures[777])

plt.title(f'features de {a}')
plt.xlabel('np.mean')
plt.ylabel('valor')
plt.grid()
plt.show()

In [ ]:
xfeatures_train, xfeatures_test = train_test_split(Xfeatures)

xfeatures_train.shape, xfeatures_test.shape

In [ ]:
xfeatures_train[0].shape, xfeatures_test[0].shape

## Autoencoder

In [ ]:
K.backend.clear_session()

input_sig = K.Input(batch_shape=(None,120,1))
x = layers.Conv1D(8,3, activation='relu', padding='same',dilation_rate=2)(input_sig)
x1 = layers.MaxPooling1D(2)(x)
x2 = layers.Conv1D(4,3, activation='relu', padding='same',dilation_rate=2)(x1)
x3 = layers.MaxPooling1D(2)(x2)
x4 = layers.AveragePooling1D()(x3)
flat = layers.Flatten()(x4)
encoded = layers.Dense(20)(flat)

d1 = layers.Dense(60)(encoded)
d2 = layers.Reshape((15,4))(d1)
d3 = layers.Conv1D(4,1,strides=1, activation='relu', padding='same')(d2)
d4 = layers.UpSampling1D(2)(d3)
d5 = layers.Conv1D(8,1,strides=1, activation='relu', padding='same')(d4)
d6 = layers.UpSampling1D(2)(d5)
d7 = layers.UpSampling1D(2)(d6)
decoded = layers.Conv1D(1,1,strides=1, activation='sigmoid', padding='same')(d7)

autoencoder = K.Model(input_sig, decoded, name='autoencoder')

autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()

In [ ]:
%%time

with tf.device(PHYS_DEVICE_NAME):
    K.backend.clear_session()
    history = autoencoder.fit(x=xfeatures_train,
                              y=xfeatures_train,
                              epochs=20,
                              batch_size=20,
                              shuffle=True,
                              validation_data=(xfeatures_test, xfeatures_test)
                             )

In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'val'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.grid()
plt.show()

In [ ]:
# K.backend.clear_session()

# input_sig = K.Input(batch_shape=(None,xtrain.iloc[0].shape[0],1)) # 120x1

# e = layers.Conv1D(30,4, activation='relu', padding='same')(input_sig)
# __pool = 2
# e = layers.AveragePooling1D(__pool)(e)
# f = layers.Flatten()(e)
# print(e.shape)
# d = layers.Reshape((e.shape[1],e.shape[2]))(f)
# d = layers.UpSampling1D(__pool)(d)
# d = layers.Conv1D(1,1,strides=1, activation='linear', padding='same')(d)

# autoencoder = K.Model(input_sig, d, name='autoencoder')
# autoencoder.compile(optimizer='adam', loss='mse')
# autoencoder.summary()

# K.backend.clear_session()
# history = autoencoder.fit(x=xtrain,
#                           y=xtrain,
#                           epochs=20,
#                           batch_size=10,
#                           shuffle=True,
#                           validation_data=(xtest, xtest)
#                          )

# plt.figure(figsize=(12, 6))

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.legend(['train', 'val'], loc='upper right')
# plt.xlabel('epoch')
# plt.ylabel('loss')

# plt.grid()
# plt.show()

In [ ]:
K.backend.clear_session()

input_sig = K.Input(batch_shape=(None,xtrain.iloc[0].shape[0],1)) # 120x1

e = layers.Conv1D(30,4, activation='relu', padding='same')(input_sig)
__pool = 2
e = layers.AveragePooling1D(__pool)(e)
f = layers.Flatten()(e)
print(e.shape)
d = layers.Reshape((e.shape[1],e.shape[2]))(f)
d = layers.UpSampling1D(__pool)(d)
d = layers.Conv1D(1,1,strides=1, activation='linear', padding='same')(d)


autoencoder = K.Model(input_sig, d, name='autoencoder')
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

K.backend.clear_session()
history = autoencoder.fit(x=xtrain,
                          y=xtrain,
                          epochs=20,
                          batch_size=10,
                          shuffle=True,
                          validation_data=(xtest, xtest)
                         )

plt.figure(figsize=(12, 6))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'val'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.grid()
plt.show()

## Classificador

In [ ]:
Xvivae = df[df['dataset'] == 'vivae'].values[:,4:].astype('float32')
Xverbo = df[df['dataset'] == 'verbo'].values[:,4:].astype('float32')

Xverbo.shape, Xvivae.shape

## Decomposição

___
Refs:

- [62] https://sci-hub.se/10.1109/tmm.2017.2766843
- [63] https://sci-hub.se/10.1016/j.bspc.2018.08.035
- [64] https://arxiv.org/pdf/1712.08708.pdf
- [65] https://sci-hub.se/10.1109/icassp.2018.8462685
- [66] https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9136653
- [67] https://sci-hub.se/10.1109/ectidamtncon48261.2020.9090768
- [68] https://www.isca-speech.org/archive_v0/Interspeech_2019/pdfs/2561.pdf
- https://sci-hub.se/10.7717/peerj.488
